In [17]:
import pandas as pd

In [18]:
df=pd.read_csv('Walmart_sales.csv')

In [19]:
df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,05-02-2010,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,12-02-2010,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,19-02-2010,1611968.17,0,39.93,2.514,211.289143,8.106
3,1,26-02-2010,1409727.59,0,46.63,2.561,211.319643,8.106
4,1,05-03-2010,1554806.68,0,46.50,2.625,211.350143,8.106


In [20]:
df['Year']=df['Date'].str.split('-').str[2].astype(int)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6435 entries, 0 to 6434
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         6435 non-null   int64  
 1   Date          6435 non-null   object 
 2   Weekly_Sales  6435 non-null   float64
 3   Holiday_Flag  6435 non-null   int64  
 4   Temperature   6435 non-null   float64
 5   Fuel_Price    6435 non-null   float64
 6   CPI           6435 non-null   float64
 7   Unemployment  6435 non-null   float64
 8   Year          6435 non-null   int32  
dtypes: float64(5), int32(1), int64(2), object(1)
memory usage: 427.4+ KB


In [22]:
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')

In [23]:
df['Holiday_Flag']=df['Holiday_Flag'].astype(bool)

In [24]:
df['CPI']=df['CPI'].round(2)
df['Fuel_Price']=df['Fuel_Price'].round(2)


In [25]:
import clickhouse_connect

In [26]:
con=clickhouse_connect.get_client(host='localhost', port='8123',
                                  username='default')

In [27]:
# con.command('create database third_kt')

In [28]:
# con.command('drop table third_kt.walmart_sales_table')

In [29]:
ct='''
create table third_kt.walmart_sales_table 
(
    Store Int64,
    Date date,
    Year Int64,
    Weekly_Sales Float64,
    Holiday_Flag Bool,
    Temperature Float64,
    Fuel_Price Float64,
    CPI Float64,
    Unemployment Float64
)
engine = MergeTree()
order by Store
'''
con.command(ct)
con.insert_df('third_kt.walmart_sales_table',df)

In [49]:
first_t=''' 
create table third_kt.year_sum
(
Year Int64,
Store Int64,
Sales Float64
)
engine=SummingMergeTree()
order by (Year, Store);
'''
ins='''
insert into third_kt.year_sum(Year,Store,Sales)
select Year,Store,Weekly_Sales
from third_kt.walmart_sales_table
'''
con.command(first_t)
con.query(ins)

In [40]:
# con.command('drop table third_kt.year_sum')

In [54]:
view=''' 
create view if not exists third_kt.sum_year as
select *
from third_kt.year_sum'''
con.query(view)

In [55]:
view2=''' 
create materialized view third_kt.best_revenue
engine = AggregatingMergeTree()
order by Store
populate
as
select
    toStartOfWeek(Date) AS Week_Start,
    Store,
    sum(Weekly_Sales) AS Weekly_Revenue,
    avg(Temperature) AS Avg_Temperature,
    avg(Fuel_Price) AS Avg_Fuel_Price
from third_kt.walmart_sales_table
group by Week_Start, Store
'''
con.query(view2)